# Importacion de librerias

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
random.seed(42)

pd.options.display.float_format = '{:.5f}'.format



# Obtencion de datos

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:
tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path and symbol.spread < 15]
tickers

['AAPLm',
 'ABBVm',
 'ABTm',
 'ADBEm',
 'ADPm',
 'AMDm',
 'AMGNm',
 'AMTm',
 'ATVIm',
 'BAm',
 'BABAm',
 'BACm',
 'BIIBm',
 'BMYm',
 'CHTRm',
 'CMCSAm',
 'CMEm',
 'COSTm',
 'CSCOm',
 'CSXm',
 'CVSm',
 'EAm',
 'EBAYm',
 'EQIXm',
 'Fm',
 'FBm',
 'GILDm',
 'GOOGLm',
 'HDm',
 'IBMm',
 'INTCm',
 'JNJm',
 'JPMm',
 'KOm',
 'LINm',
 'LLYm',
 'LMTm',
 'MAm',
 'MCDm',
 'MDLZm',
 'MMMm',
 'MOm',
 'MRKm',
 'MSm',
 'NFLXm',
 'NKEm',
 'ORCLm',
 'PEPm',
 'PFEm',
 'PGm',
 'PMm',
 'REGNm',
 'SBUXm',
 'Tm',
 'TMOm',
 'TMUSm',
 'TSLAm',
 'UNHm',
 'UPSm',
 'Vm',
 'VZm',
 'WFCm',
 'WMTm',
 'XOMm',
 'AMCm',
 'BBm',
 'BBBYm',
 'BEKEm',
 'BIDUm',
 'BILIm',
 'BRQSm',
 'BYNDm',
 'CANm',
 'EDUm',
 'FTNTm',
 'FUTUm',
 'IQm',
 'JDm',
 'NIOm',
 'NTESm',
 'RLXm',
 'TALm',
 'TIGRm',
 'TMEm',
 'VIPSm',
 'XPEVm',
 'YUMCm',
 'ZTOm',
 'METAm']

In [6]:

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2014, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

timeframes = [
    # mt5.TIMEFRAME_M15,
    # mt5.TIMEFRAME_M30,
    # mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H4,
]

symbols = get_data(tickers, timeframes, date_from, date_to)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AAPLm
ABBVm
ABTm
ADBEm
ADPm
AMDm
AMGNm
AMTm
ATVIm
BAm
BABAm
BACm
BIIBm
BMYm
CHTRm
CMCSAm
CMEm
COSTm
CSCOm
CSXm
CVSm
EAm
EBAYm
EQIXm
Fm
FBm
GILDm
GOOGLm
HDm
IBMm
INTCm
JNJm
JPMm
KOm
LINm
LLYm
LMTm
MAm
MCDm
MDLZm
MMMm
MOm
MRKm
MSm
NFLXm
NKEm
ORCLm
PEPm
PFEm
PGm
PMm
REGNm
SBUXm
Tm
TMOm
TMUSm
TSLAm
UNHm
UPSm
Vm
VZm
WFCm
WMTm
XOMm
AMCm
BBm
BBBYm
BEKEm
BIDUm
BILIm
BRQSm
BYNDm
CANm
EDUm
FTNTm
FUTUm
IQm
JDm
NIOm
NTESm
RLXm
TALm
TIGRm
TMEm
VIPSm
XPEVm
YUMCm
ZTOm
METAm


In [7]:
max_start_date = None
intervals_start_dates = {}

for interval in timeframes:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            print(ticker)
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates

AAPLm


{16388: Timestamp('2021-10-27 16:00:00')}

In [9]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00')
limit_date_train

Timestamp('2022-10-01 00:00:00')

# Test preliminar

In [10]:
from backbone.utils.general_purpose import calculate_units_size, diff_pips


class BbandsCross(Strategy):
    risk = 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200

    atr_multiplier = 4
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )

        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)


    def next(self):
        actual_close = self.data.Close[-1]
        
        if self.position:
            if self.position.is_long:
                if crossover(self.data.Close, self.middle_band):
                    self.position.close()

            if self.position.is_short:
                if crossover(self.middle_band, self.data.Close):
                    self.position.close()

        else:

            if crossover(self.data.Close, self.lower_band) and actual_close > self.sma[-1]:
                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.buy(
                    size=units,
                    sl=sl_price
                )
                
            if crossover(self.upper_band, self.data.Close) and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.sell(
                    size=units,
                    sl=sl_price
                )

In [11]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    BbandsCross, 
]

experiments = parameter_combinations = list(itertools.product(
    tickers, timeframes, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    start_date = intervals_start_dates[interval]

    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        strategy,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

performance.drawdown_duration = pd.to_timedelta(performance.drawdown_duration)

performance.drawdown_duration = performance.drawdown_duration.dt.days

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


In [13]:

filter_performance = performance[
    (performance['return']>0) 
].sort_values(by=['return/dd'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
timeframes = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BbandsCross,CSXm,16388,0.73044,0.47299,100472.99405,0.00649,220,100.00000,1.44999,1,2.07003,0.91075,100472.99405,338 days,72.89296,0.32574
0,BbandsCross,KOm,16388,0.89416,1.05338,101053.37547,0.03734,127,100.00000,2.20975,4,1.49184,2.36794,101053.37547,338 days,28.21409,1.63432
0,BbandsCross,ABTm,16388,0.83001,2.75151,102751.50861,0.23753,12,100.00000,2.62084,6,2.56195,8.56102,102751.50861,338 days,11.58396,4.32652
0,BbandsCross,UPSm,16388,0.61386,1.54271,101542.70951,0.17482,4,100.00000,1.75028,2,4.89914,2.18579,101542.70951,338 days,8.82479,1.44264
0,BbandsCross,ZTOm,16388,0.73797,1.13139,101131.39490,0.17847,1,100.00000,1.76248,2,5.83413,1.45719,101131.39490,338 days,6.33958,1.05473
0,BbandsCross,Tm,16388,0.73776,4.87652,104876.52326,1.08101,134,66.66667,0.89677,3,5.24650,8.01457,104876.52326,338 days,4.51110,3.24857
0,BbandsCross,CHTRm,16388,0.72667,1.96185,101961.84806,0.45639,7,100.00000,1.34738,3,4.57796,8.74317,101961.84806,338 days,4.29859,1.86742
0,BbandsCross,METAm,16388,0.78250,4.58950,104589.49730,1.20783,111,83.33333,1.37755,6,5.64023,11.77977,104589.49730,338 days,3.79980,4.04504
0,BbandsCross,FBm,16388,0.78287,4.58754,104587.54077,1.20852,111,83.33333,1.37678,6,5.63808,11.76471,104587.54077,338 days,3.79600,4.04205
0,BbandsCross,MRKm,16388,0.57472,0.93550,100935.49618,0.28219,49,100.00000,1.46348,2,2.76405,2.73224,100935.49618,338 days,3.31518,0.80156


In [14]:
for ticker, interval in zip(portfolio[:3], timeframes[:3]):
    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        BbandsCross,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )

    stats = bt_train.run()
    bt_train.plot(filename=f'./{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\

# WFO

In [ ]:
list(np.arange(1, 5, 0.3).round(2))

In [ ]:

from backtest.utils import plot_full_equity_curve, walk_forward

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    

stats_per_symbol = {}
lookback_bars = 1000
validation_bars = 250
warmup_bars = 200

wfo_performance = pd.DataFrame()
for ticker, interval in zip(portfolio, timeframes):
    
    # print(ticker, interval)

    params = {
        'bband_std' : list(np.arange(1, 3, 0.2)),
        'atr_multiplier':list(np.arange(1, 5, 0.3).round(2)),
        'maximize': optim_func_2
    }
    
    fracc_df = symbols[ticker][interval] * 0.01

    wfo_stats = walk_forward(
        BbandsCross,
        fracc_df, 
        lookback_bars=lookback_bars,
        validation_bars=validation_bars,
        warmup_bars=warmup_bars, 
        params=params,
        commission=7e-4, 
        margin=1/30, 
        cash=10_000,
        verbose=False
    )
    
    stats_per_symbol[ticker] = {}
    stats_per_symbol[ticker][interval] = wfo_stats
    
    df_equity = wfo_stats['_equity']
    
    plot_full_equity_curve(df_equity, title=f'{ticker}, {interval}')
    
    # Calculo el stability ratio
    x = np.arange(df_equity.shape[0]).reshape(-1, 1)
    reg = LinearRegression().fit(x, df_equity.Equity)
    stability_ratio = reg.score(x, df_equity.Equity)

    # Extraigo metricas
    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[wfo_stats['Return [%]']],
        'drawdown':[wfo_stats['Max. Drawdown [%]']],
        'final_eq':[wfo_stats['Equity Final [$]']],
        'drawdown_duration':[wfo_stats['Max. Drawdown Duration']],
        'win_rate':[wfo_stats['Win Rate [%]']], 
        'sharpe_ratio':[wfo_stats['Sharpe Ratio']],
        'trades':[wfo_stats['# Trades']],
        'avg_trade_percent':[wfo_stats['Avg. Trade [%]']],
        'exposure':[wfo_stats['Exposure Time [%]']],
        'final_equity':[wfo_stats['Equity Final [$]']],
        'Duration':[wfo_stats['Duration']],

    })

    wfo_performance = pd.concat([wfo_performance, df_stats])

wfo_performance['return/dd'] = wfo_performance['return'] / -wfo_performance['drawdown']
wfo_performance['drawdown'] = -wfo_performance['drawdown']
wfo_performance['custom_metric'] = (wfo_performance['return'] / (1 + wfo_performance.drawdown)) * np.log(1 + wfo_performance.trades)

wfo_performance.drawdown_duration = pd.to_timedelta(wfo_performance.drawdown_duration)
wfo_performance.drawdown_duration = wfo_performance.drawdown_duration.dt.days

wfo_performance.sort_values(by='return/dd', ascending=False)

In [ ]:
best_performers = wfo_performance[wfo_performance['custom_metric'] > 1]
best_performers

# Montecarlo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import montecarlo_statistics_simulation


# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []

dd_df = pd.DataFrame()
ret_df = pd.DataFrame()

for index, row in best_performers.iterrows():
    print(f"Procesando ticker: {row.ticker}")
    trades_history = stats_per_symbol[row.ticker][row.interval]._trades
    
    dd = pd.DataFrame()
    ret = pd.DataFrame()
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, dd, ret = montecarlo_statistics_simulation(trade_history=trades_history, n_simulations=100_000, initial_equity=100_000, threshold_ruin=0.8, return_raw_curves=True)
    
    display(mc)
    
    dd['ticker'] = row.ticker
    ret['ticker'] = row.ticker
    
    dd_df = pd.concat([dd_df, dd])
    ret_df = pd.concat([ret_df, ret])



In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 8))
sns.boxplot(data=dd_df, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = dd_df["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = dd_df["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(25, 8))
sns.boxplot(data=ret_df, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = ret_df["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = ret_df["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
